In [1]:
import requests
import random

# The URL of the endpoint you want to send a request to
base_url = 'http://localhost:8080/'

# Get users from /users
url = base_url + 'users'
response = requests.get(url)
harvey_mudd_dorm_names = ['Atwood', 'Case', 'East', 'Linde', 'North', 'South', 'West', 'Sontag', 'Drinkward']

users = None
# Check the response
if response.status_code == 200:
    print('Successfully got users!')
    print(response.json())  # If response is in JSON format
    users = response.json()

if response.status_code == 404:
    print('Failed with status code:', response.status_code)
    exit(1)

# Get rooms from /rooms
url = base_url + 'rooms'
response = requests.get(url)
req_rooms = None

# Check the response
if response.status_code == 200:
    print('Successfully got rooms!')
    print(response.json())  # If response is in JSON format
    req_rooms = response.json()

if response.status_code == 404:
    print('Failed with status code:', response.status_code)
    exit(1)

dorms = {}

for dorm in harvey_mudd_dorm_names:
    dorms[dorm] = []
    url = base_url + 'rooms/simple/' + dorm
    response = requests.get(url)

    # Check the response
    if response.status_code == 200:
        print("Successfully got rooms for dorm " + dorm + "!")
        # print(response.json())
        dorms[dorm] = response.json()

    if response.status_code == 404:
        print('Failed with status code:', response.status_code)
        exit(1)

# in golang the structure of the dorm from simple route is as follows:
# type DormSimple struct {
# 	DormName    string        `json:"dormName"`
# 	Description string        `json:"description"`
# 	Floors      []FloorSimple `json:"floors"`
# }

# type FloorSimple struct {
# 	FloorNumber int           `json:"floorNumber"`
# 	Suites      []SuiteSimple `json:"suites"`
# }

# type SuiteSimple struct {
# 	Rooms       []RoomSimple `json:"rooms"`
# 	SuiteDesign string       `json:"suiteDesign"`
# 	SuiteUUID   uuid.UUID    `json:"suiteUUID"`
# }

# type RoomSimple struct {
# 	RoomNumber   string       `json:"roomNumber"`
# 	MaxOccupancy int          `json:"maxOccupancy"`
# 	PullPriority PullPriority `json:"pullPriority"`
# 	RoomUUID     uuid.UUID    `json:"roomUUID"`
# 	Occupant1    int          `json:"occupant1"`
# 	Occupant2    int          `json:"occupant2"`
# 	Occupant3    int          `json:"occupant3"`
# 	Occupant4    int          `json:"occupant4"`
# }
# data = {
    
# }

# # Send a POST request
# response = requests.post(url, json=data)

# # Send a GET request with parameters
# # response = requests.get(url, params=data)

# # Check the response
# if response.status_code == 200:
#     print('Success!')
#     print(response.json())  # If response is in JSON format
# else:
#     print('Failed with status code:', response.status_code)

Successfully got users!
[{'Id': 261, 'Year': 'junior', 'FirstName': 'Anna', 'LastName': 'Meyer', 'Email': '', 'DrawNumber': 130, 'Preplaced': True, 'InDorm': 0, 'SGroupUUID': '00000000-0000-0000-0000-000000000000', 'Participated': False, 'RoomUUID': '00000000-0000-0000-0000-000000000000'}, {'Id': 21, 'Year': 'senior', 'FirstName': 'Nellie', 'LastName': 'Hayes', 'Email': '', 'DrawNumber': 103, 'Preplaced': True, 'InDorm': 0, 'SGroupUUID': '00000000-0000-0000-0000-000000000000', 'Participated': False, 'RoomUUID': '00000000-0000-0000-0000-000000000000'}, {'Id': 30, 'Year': 'senior', 'FirstName': 'Martha', 'LastName': 'Freeman', 'Email': '', 'DrawNumber': 95, 'Preplaced': False, 'InDorm': 0, 'SGroupUUID': '00000000-0000-0000-0000-000000000000', 'Participated': False, 'RoomUUID': '00000000-0000-0000-0000-000000000000'}, {'Id': 33, 'Year': 'senior', 'FirstName': 'Jean', 'LastName': 'Berger', 'Email': '', 'DrawNumber': 82, 'Preplaced': False, 'InDorm': 1, 'SGroupUUID': '00000000-0000-0000-000

In [27]:

# print(dorms)
used_suites = {}
used_users = {}
# pick 3 random suites from each dorm
for i in range(3):
    for dormName in dorms:
        for floor in dorms[dormName]['floors']:
            dorm = dorms[dormName]
            print("Dorm: " + dormName)
            suite = random.choice(floor['suites'])
            print("Suite: " + str(suite))
            print("Suite UUID: " + suite['suiteUUID'])
            if suite['suiteUUID'] in used_suites:
                continue
            used_suites[suite['suiteUUID']] = True
            # pick a random room to be the suite leader
            try:
                room = random.choice(suite['rooms'])
            except:
                continue
        
            print("Leader Room: " + str(room))

            if room['maxOccupancy'] != 1:
                # pick some users and assign them to the room
                # pick a random number of users to assign to the room
                num_users = random.randint(1, room['maxOccupancy'])

                # pick random users to assign to the room
                # make sure the users are not already assigned to a room
                users_to_assign = []
                for j in range(num_users):
                    user = random.choice(users)
                    while user['Id'] in used_users:
                        user = random.choice(users)
                    used_users[user['Id']] = True
                    users_to_assign.append(user)
                
                # assign the users to the room with the following request body:
                # curl -X PATCH http:/localhost:8080/rooms/2c4a04bb-ddd6-4f02-ba55-a04aeb5d59f3 \
                #     -H "Content-Type: application/json" \
                #     -d '{"proposedOccupants": [9], "pullType": 1}'
                url = base_url + 'rooms/' + str(room['roomUUID'])
                data = {
                    "proposedOccupants": [user['Id'] for user in users_to_assign],
                    "pullType": 1
                }
                response = requests.patch(url, json=data)

                # Check the response
                if response.status_code == 200:
                    print('Successfully assigned users to room!')
                    print(response.json())
                else:
                    print('Failed with status code:', response.status_code)
            else:
                # pick a random user to assign to the room
                user = random.choice(users)
                while user['Id'] in used_users:
                    user = random.choice(users)
                used_users[user['Id']] = True

                # assign the user to the room with the following request body:
                # curl -X PATCH http:/localhost:8080/rooms/2c4a04bb-ddd6-4f02-ba55-a04aeb5d59f3 \
                #     -H "Content-Type: application/json" \
                #     -d '{"proposedOccupants": [9], "pullType": 1}'
                url = base_url + 'rooms/' + str(room['roomUUID'])
                data = {
                    "proposedOccupants": [user['Id']],
                    "pullType": 1
                }
                response = requests.patch(url, json=data)

                room_leader_uuid = room['roomUUID']

                # Check the response
                if response.status_code == 200:
                    print('Successfully assigned user to room!')
                    print(response.json())
                else:
                    print('Failed with status code:', response.status_code)

                
                
                try:
                    # pick a random room to be the suite leader
                    room = random.choice(suite['rooms'])
                except:
                    continue
                print("Pulled Room: " + str(room))

                if room['maxOccupancy'] != 1:
                    continue
                else:
                    # pick a random user to assign to the room
                    user = random.choice(users)
                    while user['Id'] in used_users:
                        user = random.choice(users)
                    used_users[user['Id']] = True

                    # assign the user to the room with the following request body:
                    # curl -X PATCH http:/localhost:8080/rooms/239f4ef3-052d-48fa-ac0f-a57900d87324 \
                    # -H "Content-Type: application/json" \
                    # -d '{"proposedOccupants": [8, 13, 14], "pullType": 2, "pullLeaderRoom": "2c4a04bb-ddd6-4f02-ba55-a04aeb5d59f3"}'
                    url = base_url + 'rooms/' + str(room['roomUUID'])
                    data = {
                        "proposedOccupants": [user['Id']],
                        "pullType": 2,
                        "pullLeaderRoom": room_leader_uuid
                    }
                    response = requests.patch(url, json=data)

                    # Check the response
                    if response.status_code == 200:
                        print('Successfully assigned user to room!')
                        print(response.json())
                    else:
                        print('Failed with status code:', response.status_code)
            




Dorm: Atwood
Suite: [{'rooms': [{'roomNumber': '109', 'maxOccupancy': 1, 'pullPriority': {'valid': False, 'isPreplaced': False, 'hasInDorm': False, 'drawNumber': 0, 'year': 0, 'pullType': 0, 'inherited': {'hasInDorm': False, 'drawNumber': 0, 'year': 0}}, 'roomUUID': '8a23071f-53ff-48a4-9570-a674e381f6d9', 'occupant1': 0, 'occupant2': 0, 'occupant3': 0, 'occupant4': 0}, {'roomNumber': '101', 'maxOccupancy': 2, 'pullPriority': {'valid': False, 'isPreplaced': False, 'hasInDorm': False, 'drawNumber': 0, 'year': 0, 'pullType': 0, 'inherited': {'hasInDorm': False, 'drawNumber': 0, 'year': 0}}, 'roomUUID': '66c01408-1607-4cdb-a595-3b1d4912a605', 'occupant1': 0, 'occupant2': 0, 'occupant3': 0, 'occupant4': 0}, {'roomNumber': '111', 'maxOccupancy': 1, 'pullPriority': {'valid': False, 'isPreplaced': False, 'hasInDorm': False, 'drawNumber': 0, 'year': 0, 'pullType': 0, 'inherited': {'hasInDorm': False, 'drawNumber': 0, 'year': 0}}, 'roomUUID': '00e0a6b8-df29-4c4b-96fe-85616bd67710', 'occupant1': 

TypeError: list indices must be integers or slices, not str